In [1]:
import pandas as pd
import requests
from config import nps_key,weather_key
pd.options.mode.chained_assignment = None

# Extract

In [9]:
# National Parks API Documentation : https://www.nps.gov/subjects/developer/api-documentation.htm

url = "https://developer.nps.gov/api/v1/parks?"
park = []
params = {'api_key':nps_key,
# 'stateCode':stateCode,
'q':"hiking",
'limit':300}
response = requests.get(url,params=params).json()
park=[]
for x in range(len(response['data'])):

            park_name = response['data'][x]['fullName']
            parkCode = response['data'][x]['parkCode']
            
            street_add = response['data'][x]['addresses'][0]['line1']
            city  = response['data'][x]['addresses'][0]['city']
            state = response['data'][x]['addresses'][0]['stateCode']
            zip_code = response['data'][x]['addresses'][0]['postalCode']
            email = response['data'][x]['contacts']['emailAddresses'][0]['emailAddress']
            lat = response['data'][x]['latitude']
            lng = response['data'][x]['longitude']
            weather_info = response['data'][x]['weatherInfo']
            try:
                fee_car = response['data'][x]['entranceFees'][0]['cost']
                fee_person = response['data'][x]['entranceFees'][2]['cost']
                phone = response['data'][x]['contacts']['phoneNumbers'][0]['phoneNumber']
            except:
                fee_car = 0
                fee_person = 0
                phone = "N/A"
            data_dict = {'park_code':parkCode,
                'park_name': park_name,
            'fee_per_car':fee_car,
            'fee_per_person':fee_person,
            'street_address':street_add,
            'city':city,
            'state':state,
            'zip_code':zip_code,
            'phone_number':phone,
            'email_address':email,
            'lat':lat,
            'lng':lng,
            'Weather Summary':weather_info}
            park.append(data_dict)
park_df=pd.DataFrame(park)

In [5]:
weather_df = park_df[['park_code','park_name','city','state','lat','lng']]
weather_df["Min_Temp"]=""
weather_df["Max_Temp"]=""
weather_df["Humidity"]=""
weather_df["Cloudiness"]=""
weather_df["Wind_Speed"]=""
weather_df["Date"]=""


units = 'imperial'
weather_key=weather_key
openweather =  "https://api.openweathermap.org/data/2.5/weather?"

print("Beginning Data Retrieval")
print("-----------------------------")


for index,row in weather_df.iterrows():
    park = row["park_name"]
    lat = row['lat']
    lng = row['lng']
    params = {'appid':weather_key,
             'lat':lat,
             'lon':lng}
    response = requests.get(openweather,params=params).json()
    try:
        min_temp = response["main"]["temp_min"]
        max_temp = response["main"]["temp_max"]
        humidity = response["main"]["humidity"]
        wind = response["wind"]["speed"]
        country = response["sys"]["country"]
        date = response["dt"]
        clouds = response['clouds']['all']
        weather_df.loc[index,"Humidity"]=humidity
        weather_df.loc[index,"Min_Temp"]=min_temp
        weather_df.loc[index,"Max _Temp"]=max_temp
        weather_df.loc[index,"Humidity"]=humidity
        weather_df.loc[index,"Cloudiness"]=clouds
        weather_df.loc[index,"Wind_Speed"]=wind
        weather_df.loc[index,"Country"]=country
        weather_df.loc[index,"Date"]=date
    except:
        pass

Beginning Data Retrieval
-----------------------------


In [17]:
weather_df.to_csv('cleaned/weather_data.csv')

# Transform

In [10]:
park_df = park_df.replace({'Hawaiʻi Volcanoes National Park':'Hawaii Volcanoes National Park','Haleakalā National Park':'Haleakala National Park','Tumacácori National Historical Park':'Tumacacori National Historical Park'})

In [11]:
# All Trails pulled from 
# https://www.kaggle.com/code/jessicaqjiang/all-trails-in-np-analysis/data
all_trails = pd.read_csv('Working/Resources/AllTrails data - nationalpark.csv')
all_trails=all_trails.reset_index(drop=True)
park_code = park_df[["park_name","park_code"]]

In [12]:
# Some National Parks are listed differently in the All Trails Dataset; need to rename
all_trails = all_trails.replace({'Black Canyon of the Gunnison National Park':'Black Canyon Of The Gunnison National Park',
                          'Congaree National Park Wilderness':'Congaree National Park,','Denali National Park':'Denali National Park & Preserve',
                          'Glacier Bay National Park':'Glacier Bay National Park & Preserve','Great Sand Dunes National Park and Preserve':'Great Sand Dunes National Park & Preserve',
                          'Katmai National Park':'Katmai National Park & Preserve','Kings Canyon National Park':'Sequoia & Kings Canyon National Parks',
                          'Sequoia National Park':'Sequoia & Kings Canyon National Parks','Redwood National Park':'Redwood National and State Parks'})

In [13]:
# Inserting Park Code as a sort of Foreign Key
all_trails['park_code']=''

for index,row in all_trails.iterrows():
    park = row['area_name']
    try:
        code=park_code.loc[(park_code['park_name']==park),['park_code']].values[0][0]
        all_trails.iloc[index,18]=code
    except:
        all_trails.iloc[index,18]='look'

In [14]:
# Some national parks in All Trails do not appear in the National Parks API; filter out and save off
look_df=all_trails.loc[all_trails['park_code']=='look']
look_df.to_csv('cleaned/trails_without_code.csv')

In [15]:
all_trails_cleaned = all_trails.loc[all_trails['park_code']!='look',:]
# Rename the column with trail name
all_trails_cleaned.rename(columns= {"name": "trail"}, inplace = True)

# Drop the unwanted columns
# Upon investigation all units are in meters regardless of the input in the units column
all_trails_cleaned = all_trails_cleaned.drop(['country_name','visitor_usage','units'], axis = 1)

# Make the trail_id the index
all_trails_cleaned=all_trails_cleaned.set_index('trail_id')

In [16]:
alltrails_filtered =all_trails_cleaned[all_trails_cleaned['avg_rating'] >= 2]
alltrails_filtered = all_trails_cleaned[all_trails_cleaned['num_reviews'] >= 10]
alltrails_filtered.sort_values(by='popularity', inplace=True, ascending=False)
alltrails_filtered.to_csv('cleaned/trails.csv')

In [18]:
summary_table = pd.DataFrame({'num_of_trails':alltrails_filtered.groupby('park_code').count()['trail'],
                'avg_difficulty':alltrails_filtered.groupby('park_code').mean()['difficulty_rating'],
                'avg_rating':alltrails_filtered.groupby('park_code').mean()['avg_rating']})

summary_table = summary_table.reset_index(drop=False)

In [19]:
aggregate_park = pd.merge(park_df,summary_table,how = 'left', on='park_code')
aggregate_park = aggregate_park.fillna(0)
aggregate_park.to_csv('cleaned/National_Parks.csv',index=False)

# Load

In [147]:
from sqlalchemy import create_engine
# from config import pg_key

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = pg_key
host = 'localhost'
port = 5432

In [ ]:
database_name = 'project_2_group_4_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
print(rds_connection_string)

In [ ]:
alltrails_filtered.to_sql(name='alltrails_filtered', con=engine, if_exists='append', index=False)

In [ ]:
weather_data_df.to_sql(name='weather_data', con=engine, if_exists='append', index=False)

In [ ]:
aggregate_park.to_sql(name='national_parks_cleaned', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from alltrails_filtered', con=engine).head()

In [ ]:
pd.read_sql_query('select * from national_parks_cleaned', con=engine).head()

In [ ]:
pd.read_sql_query('select * from weather_data', con=engine).head()